# Loading model

**‼️  Import mediapipe before tesorflow due to runtime issue**


In [55]:
import mediapipe as mp
import tensorflow as tf

In [56]:
import os

In [57]:
model_filename = "keypoint_classifier_new.hdf5"
model_save_path = os.path.join('model', model_filename)

In [58]:
DIRECTION = {
    'up': 0,
    'down': 1,
    'back': 2,
    'forward': 3,
    'left': 4,
    'right': 5
}
INVERSE_DIRECTION = {val: key for key, val in DIRECTION.items()}

In [59]:
model = tf.keras.models.load_model(model_save_path)

In [60]:
model.weights

[<tf.Variable 'dense_8/kernel:0' shape=(42, 32) dtype=float32, numpy=
 array([[-0.0761361 ,  0.26863056, -0.19491455, ..., -0.17018458,
          0.38613313, -0.48222455],
        [-0.23871379,  0.35555923, -0.05818052, ..., -0.17977929,
          0.4600649 ,  0.12289268],
        [ 0.01607633,  0.7967301 , -0.8055061 , ...,  0.05076262,
         -0.282856  , -0.3635127 ],
        ...,
        [ 0.19674361,  0.88568705,  0.16584618, ...,  0.04952052,
          0.46894625, -0.5908134 ],
        [-0.17839509, -0.24517229,  0.6420435 , ..., -0.02508712,
          0.9631354 , -0.25712705],
        [-0.1932533 ,  0.81590784,  0.08562319, ...,  0.09486207,
         -0.1286373 , -0.553106  ]], dtype=float32)>,
 <tf.Variable 'dense_8/bias:0' shape=(32,) dtype=float32, numpy=
 array([ 0.00000000e+00, -2.20406681e-01,  1.13456666e-01, -3.90865607e-04,
         3.42744619e-01, -1.57021776e-01, -1.82737529e-01, -7.51253143e-02,
         1.69507012e-01,  0.00000000e+00,  0.00000000e+00,  4.12384689

# Showing live demo using cv2

In [61]:
import cv2
import numpy as np

In [62]:
def write_text_on_video(frame, direction, color=(255,255,255)):
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame,
               f"{direction}",
               (50, 50),
                font, 1, 
                color,
                2,
                cv2.LINE_4)

In [63]:
def extract_keypoints(landmark):
    if not landmark: return
    keypoints = []
    for data_point in enumerate(landmark[0].landmark):
        keypoints.extend(
            [data_point[1].x, data_point[1].y]
        )
    return keypoints
    

In [64]:
def add_hands_connection(frame, hands):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = hands.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    if results.multi_hand_landmarks:
        for num, hand in enumerate(results.multi_hand_landmarks):
            mp_drawing.draw_landmarks(
                image, hand, 
                mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(121,22,78), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2),
            )
    return image, results

In [65]:
# def predict_label(keypoints):
#     return INVERSE_DIRECTION[
#         np.argmax(np.squeeze(
#             model.predict([keypoints])
#                   ))]

def predict_label(keypoints):
    temp = np.squeeze(model.predict([keypoints]))
    print(temp)
    return INVERSE_DIRECTION[np.argmax(temp)]

In [66]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [67]:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.7) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        image, results = add_hands_connection(frame, hands)
        image = cv2.flip(image, 1)
        landmark_result = results.multi_hand_landmarks
        if landmark_result:
            keypoints = extract_keypoints(landmark_result)
            y_cap = predict_label(keypoints)
            write_text_on_video(image, y_cap)
        cv2.imshow("hand", image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)

[9.9999988e-01 2.9743008e-20 1.4628569e-07 1.2133604e-11 1.1603406e-12
 4.4386112e-14]
[9.9926823e-01 1.2046581e-15 7.6995086e-05 6.5482408e-04 1.3172409e-16
 7.6070407e-11]
[9.9996960e-01 1.2609152e-16 1.6081796e-05 1.4301154e-05 2.4209006e-16
 3.7118607e-11]
[9.9996209e-01 6.4014785e-16 1.9706780e-05 1.8254339e-05 7.2585033e-16
 9.0288291e-11]
[9.9989891e-01 8.2341775e-15 5.3881060e-05 4.7150595e-05 1.2684302e-14
 5.5624245e-11]
[1.4595732e-10 3.5503381e-10 2.3454157e-04 6.2486062e-13 9.9976546e-01
 8.3022196e-13]
[1.1658847e-10 2.6785332e-10 1.1523735e-04 3.0018707e-13 9.9988472e-01
 7.9073526e-13]
[8.7367322e-11 4.1353893e-10 1.2736676e-04 3.4244775e-13 9.9987257e-01
 1.1254308e-12]
[7.9691205e-11 4.3427739e-10 1.2283926e-04 3.5795951e-13 9.9987710e-01
 1.0315807e-12]
[1.9824673e-11 4.1152942e-10 4.5145051e-05 9.3578784e-14 9.9995482e-01
 5.4234530e-13]
[6.1092498e-03 3.0670841e-09 8.4114166e-05 4.8203142e-06 2.5674072e-11
 9.9380189e-01]
[1.5156256e-05 1.7714556e-09 3.2664707e-06 